In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Get the input and label files from CSVs

In [42]:
def get_train_test_splitted_data(label_files, input_files, global_mean, global_std, test_size=0.2, random_state=42):
    # Initialize lists to hold all sequences
    all_x_sequences = []
    all_y_sequences = []

    # Process each pair of input and label files
    for input_file, label_file in zip(input_files, label_files):
        # Load data
        input_df = pd.read_csv(input_file)
        if label_file.endswith('BORIS_method_II.csv'):
            label_df = pd.read_csv(label_file)
        else:
            label_df = pd.read_csv(label_file, dtype=str, na_values=[])    
        # Prepare features and labels
        features = (input_df.values - global_mean) / global_std
        
        if label_file.endswith('BORIS_method_II.csv'):
            labels = label_df.values / 100
        else:
            # Define the possible categories explicitly
            column_names = ['Happy', 'Sad', 'Scared', 'Disgusted', 'Surprised', 'Angry']

            # Create a OneHotEncoder with predefined categories
            encoder = OneHotEncoder(categories=[column_names], handle_unknown='ignore')

            # Fit and transform the label data
            labels = pd.DataFrame(
                encoder.fit_transform(label_df).toarray(),
                columns=encoder.get_feature_names_out()
            )

        # Ensure alignment of frames
        if features.shape[0] != labels.shape[0]:
            print(f"Mismatch in frames: {input_file}, {label_file}")
            continue
            
        # Sample sequences
        x_sequences, y_sequences = create_sequences(features, labels, SEQUENCE_LENGTH, STRIDE)

        # Append to global lists
        all_x_sequences.append(x_sequences)
        all_y_sequences.append(y_sequences)
        
    # Concatenate all sequences from all files
    all_x_sequences = np.concatenate(all_x_sequences, axis=0)
    all_y_sequences = np.concatenate(all_y_sequences, axis=0)

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        all_x_sequences, all_y_sequences, test_size=test_size, random_state=random_state
    )

    # Convert to TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    # Shuffle, batch, and prefetch
    train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, test_dataset

In [43]:
# Constants
SEQUENCE_LENGTH = 10
STRIDE = 5
BATCH_SIZE = 32
INPUT_DIM = 515  # Number of features per frame (e.g., biosignals + embeddings)
OUTPUT_DIM = 6 

In [44]:
# Helper function to create random sequences
def create_sequences(features, labels, sequence_length, stride):
    x_sequences, y_sequences = [], []
    for i in range(0, len(features) - sequence_length + 1, stride):
        x_sequences.append(features[i:i + sequence_length])
        y_sequences.append(labels[i:i + sequence_length])
    return np.array(x_sequences), np.array(y_sequences)

# Initialize lists to hold all sequences
all_x_sequences = []
all_y_sequences = []
all_features = []

sources = ["GUT", "ITU-YU", "MAAP"]
base_path = "//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/"
input_files, label_files = [], []

for source in sources:
    input_files.extend(glob.glob(os.path.join(base_path, source, '*_input.csv')))
    label_files.extend(glob.glob(os.path.join(base_path, source, '*_BORIS.csv')))

input_files.sort()
label_files.sort()

i = 0
for input_file in input_files:
    input_df = pd.read_csv(input_file)
    all_features.append(input_df.values)

# Concatenate all features from all files to compute global mean and std
all_features = np.concatenate(all_features, axis=0)

global_mean = all_features.mean(axis=0)
global_std = all_features.std(axis=0)

# Ensure no division by zero
global_std[global_std == 0] = 1

GUT_path_input = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT/*_input.csv'))
ITU_YU_path_input = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/ITU-YU/*_input.csv'))
MAAP_path_input = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/MAAP/*_input.csv'))

GUT_path_label_method_I = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT/*_BORIS_method_I.csv'))
ITU_YU_path_label_method_I = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/ITU-YU/*_BORIS_method_I.csv'))
MAAP_path_label_method_I = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/MAAP/*_BORIS_method_I.csv'))

GUT_path_label_method_II = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT/*_BORIS_method_II.csv'))
ITU_YU_path_label_method_II = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/ITU-YU/*_BORIS_method_II.csv'))
MAAP_path_label_method_II = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/MAAP/*_BORIS_method_II.csv'))

GUT_train_method_I, GUT_test_method_I = get_train_test_splitted_data(GUT_path_label_method_I, GUT_path_input, global_mean, global_std, test_size=0.3)
ITU_YU_train_method_I, ITU_YU_test_method_I = get_train_test_splitted_data(ITU_YU_path_label_method_I, ITU_YU_path_input, global_mean, global_std, test_size=0.3)
MAAP_train_method_I, MAAP_test_method_I = get_train_test_splitted_data(MAAP_path_label_method_I, MAAP_path_input, global_mean, global_std, test_size=0.3)

GUT_train_method_II, GUT_test_method_II = get_train_test_splitted_data(GUT_path_label_method_II, GUT_path_input, global_mean, global_std, test_size=0.3)
ITU_YU_train_method_II, ITU_YU_test_method_II = get_train_test_splitted_data(ITU_YU_path_label_method_II, ITU_YU_path_input, global_mean, global_std, test_size=0.3)
MAAP_train_method_II, MAAP_test_method_II = get_train_test_splitted_data(MAAP_path_label_method_II, MAAP_path_input, global_mean, global_std, test_size=0.3)

dataset_method_I = GUT_train_method_I.concatenate(ITU_YU_train_method_I).concatenate(MAAP_train_method_I)
dataset_method_II = GUT_train_method_II.concatenate(ITU_YU_train_method_II).concatenate(MAAP_train_method_II)

In [45]:
import pandas as pd
import numpy as np

all_features = []

for input_file in input_files:
    input_df = pd.read_csv(input_file)
    
    # Check if the DataFrame contains any NaN
    if input_df.isnull().values.any():
        print(f"NaN found in file: {input_file}")
    
    all_features.append(input_df.values)

In [47]:
path = r'S:\IO3-sessions\NEW STRUCTURE\de-earlyfusionthesis\Datasets'

tf.data.experimental.save(dataset_method_I ,os.path.join(path, 'train_dataset_method_I'))
tf.data.experimental.save(GUT_train_method_I, os.path.join(path, 'GUT_train_method_I'))
tf.data.experimental.save(GUT_test_method_I, os.path.join(path, 'GUT_test_method_I'))
tf.data.experimental.save(ITU_YU_train_method_I, os.path.join(path, 'ITU_YU_train_method_I'))
tf.data.experimental.save(ITU_YU_test_method_I, os.path.join(path, 'ITU_YU_test_method_I'))
tf.data.experimental.save(MAAP_train_method_I, os.path.join(path, 'MAAP_train_method_I'))
tf.data.experimental.save(MAAP_test_method_I, os.path.join(path, 'MAAP_test_method_I'))

tf.data.experimental.save(dataset_method_II ,os.path.join(path, 'train_dataset_method_II'))
tf.data.experimental.save(GUT_train_method_II, os.path.join(path, 'GUT_train_method_II'))
tf.data.experimental.save(GUT_test_method_II, os.path.join(path, 'GUT_test_method_II'))
tf.data.experimental.save(ITU_YU_train_method_II, os.path.join(path, 'ITU_YU_train_method_II'))
tf.data.experimental.save(ITU_YU_test_method_II, os.path.join(path, 'ITU_YU_test_method_II'))
tf.data.experimental.save(MAAP_train_method_II, os.path.join(path, 'MAAP_train_method_II'))
tf.data.experimental.save(MAAP_test_method_II, os.path.join(path, 'MAAP_test_method_II'))

In [48]:
path = r'S:\IO3-sessions\NEW STRUCTURE\de-earlyfusionthesis\Models'

## MODEL I

In [55]:
model_I = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(SEQUENCE_LENGTH, INPUT_DIM)),    # Input shape: (sequence_length, features)
    tf.keras.layers.LSTM(64, return_sequences=True),              # LSTM layer to capture temporal patterns
    tf.keras.layers.Dense(32, activation='relu'),                 # Dense layer to reduce dimensionality
    tf.keras.layers.Dense(OUTPUT_DIM, activation='softmax')       # Output layer with sigmoid for continuous values between 0 and 1
])

In [68]:
def mask_timestep(inputs):
    # Mask timesteps where any feature is 0
    # tf.reduce_any checks if any feature in the timestep is 0
    mask = tf.reduce_any(tf.equal(inputs, 0.0), axis=-1)  # Create mask for timesteps with any 0
    return mask

# Input Layer
inputs = tf.keras.Input(shape=(SEQUENCE_LENGTH, INPUT_DIM))

# Apply the custom masking logic
mask = tf.keras.layers.Lambda(mask_timestep)(inputs)

# Apply the mask to the input sequence (this will zero out timesteps that have missing values)
masked_inputs = tf.keras.layers.Masking(mask_value=0.0)(inputs)

# LSTM layers to process the masked inputs
lstm_output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(masked_inputs)

# Dense layers
dense_output = tf.keras.layers.Dense(32, activation='relu')(lstm_output)
output = tf.keras.layers.Dense(OUTPUT_DIM, activation='sigmoid')(dense_output)

# Build and compile the model
model_I = tf.keras.Model(inputs=inputs, outputs=output)

In [69]:
model_I.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [74]:
# Assuming dataset_method_I is your dataset (concatenated dataset)
# The dataset has input features and labels in the tuple format (input, label)

# Iterate through the dataset to extract labels
labels = []

for inputs, label in dataset_method_I:
    labels.append(label)

# Convert to a tensor or numpy array
y_train = np.concatenate(labels, axis=0)  # Stack labels into a single array


In [77]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Flatten the labels array to calculate class frequencies across the entire dataset
# Assuming `y_train` is the labels tensor with shape (num_samples, 10, 6)

y_train_flat = y_train.reshape(-1, 6)  # Flattening the labels: (num_samples * 10, 6)
y_train_classes = np.argmax(y_train_flat, axis=-1)  # Get the class labels for each timestep (0 to 5)

# Calculate class weights based on the frequency of each class
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_classes), y=y_train_classes)

# Convert class_weights into a dictionary format for fit() function
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class Weights:", class_weight_dict)

Class Weights: {0: 0.16905572901882865, 1: 29.853801169590643, 2: 24.97204751921733, 3: 496.31944444444446, 4: 248.15972222222223, 5: 192.1236559139785}


In [79]:
history = model_I.fit(dataset_method_I, epochs=50, batch_size=BATCH_SIZE, class_weight=class_weight_dict)

Epoch 1/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.0470 - loss: 72.3243
Epoch 2/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.0177 - loss: 45.7122
Epoch 3/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.0622 - loss: 90.8937
Epoch 4/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.0063 - loss: 61.1309
Epoch 5/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.0084 - loss: 36.5318
Epoch 6/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.0163 - loss: 44.3204
Epoch 7/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.0310 - loss: 40.3289
Epoch 8/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.0512 - loss: 45.5805
Epoch 9/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.0214 - loss: 76.8801
Epoch 10/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.0289 - loss: 39.2421
Epoch 11/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.0697 - loss: 65.7637
Epoch 12/50
225/225 ━━━━━━━━━

In [80]:
model_I.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 10, 515)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 10, 515)   │          0 │ input_layer_7[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, 10, 515)   │          0 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_1 (Any)         │ (None, 10)        │          0 │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 10, 128)   │    296,960 │ masking[0][0],    │
│ (Bidirectional)     │                   │            │ any_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 10, 32)    │      4,128 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 10, 6)     │        198 │ dense_12[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 903,860 (3.45 MB)

 Trainable params: 301,286 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 602,574 (2.30 MB)

In [81]:
# Evaluate the model
loss, acc = model_I.evaluate(MAAP_test_method_I, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(acc))

72/72 - 2s - 27ms/step - accuracy: 0.0013 - loss: 19.0578
Untrained model, accuracy:  0.00%


In [82]:
model_I.save(os.path.join(path, 'model_method_I/model.keras'))

## MODEL II

In [61]:
model_II = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(SEQUENCE_LENGTH, INPUT_DIM)),    # Input shape: (sequence_length, features)
    tf.keras.layers.LSTM(64, return_sequences=True),              # LSTM layer to capture temporal patterns
    tf.keras.layers.Dense(32, activation='relu'),                 # Dense layer to reduce dimensionality
    tf.keras.layers.Dense(OUTPUT_DIM, activation='sigmoid')       # Output layer with sigmoid for continuous values between 0 and 1
])

In [62]:
model_II.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [63]:
history = model_II.fit(dataset_method_II, epochs=50)

Epoch 1/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - loss: 0.0663 - mae: 0.1861
Epoch 2/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0117 - mae: 0.0421
Epoch 3/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0108 - mae: 0.0378
Epoch 4/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0104 - mae: 0.0362
Epoch 5/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0096 - mae: 0.0343
Epoch 6/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0094 - mae: 0.0339
Epoch 7/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0089 - mae: 0.0328
Epoch 8/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0085 - mae: 0.0318
Epoch 9/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0083 - mae: 0.0310
Epoch 10/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0079 - mae: 0.0297
Epoch 11/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0077 - mae: 0.0292
Epoch 12/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0075 - mae: 0.0284
Epoch 13/50
2

In [64]:
model_II.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 10, 64)         │       148,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10, 32)         │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10, 6)          │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 452,276 (1.73 MB)

 Trainable params: 150,758 (588.90 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 301,518 (1.15 MB)

In [65]:
# Evaluate the model
loss, acc = model_II.evaluate(MAAP_test_method_II, verbose=2)
print("Untrained model, coherence: {:5.2f}%".format(100 * (1-acc)))

72/72 - 3s - 44ms/step - loss: 0.0120 - mae: 0.0367
Untrained model, coherence: 96.33%


In [66]:
model_II.save(os.path.join(path, 'model_method_II/model.keras'))

## Transformer

In [87]:
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout

In [102]:
def get_positional_encoding(sequence_length, input_dim):
    """
    Generate a positional encoding matrix for the transformer model.
    This encoding is added to the input data to give the model an understanding of the order of the sequence.
    """
    position = np.arange(sequence_length)[:, np.newaxis]  # Shape: (sequence_length, 1)
    div_term = np.exp(np.arange(0, input_dim, 2) * -(np.log(10000.0) / input_dim))  # Shape: (input_dim // 2,)
    
    pos_enc = np.zeros((sequence_length, input_dim))  # Shape: (sequence_length, input_dim)
    
    # Apply sine to even indices (0, 2, 4, ...)
    pos_enc[:, 0::2] = np.sin(position * div_term)  # Apply sine to even indices (0, 2, 4, ...)
    pos_enc[:, 1::2] = np.cos(position * div_term)
    # Apply cosine to odd indices (1, 3, 5, ...)
    if input_dim % 2 != 0:  # Even input_dim
        pos_enc[:, -1] = np.sin(position * div_term[-1])  # If odd, handle the last dimension separately

    return tf.constant(pos_enc, dtype=tf.float32)



In [103]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate):
    # Multi-Head Self Attention Layer
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size, dropout=dropout_rate)(inputs, inputs)
    
    # Skip connection and normalization
    x = tf.keras.layers.Add()([inputs, attention_output])
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    
    # Feed-forward network
    ffn_output = tf.keras.layers.Dense(ff_dim, activation='relu')(x)
    ffn_output = tf.keras.layers.Dense(inputs.shape[-1])(ffn_output)
    
    # Skip connection and normalization
    x = tf.keras.layers.Add()([x, ffn_output])
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    
    return x

In [104]:
def build_transformer_model(sequence_length, input_dim, output_dim, num_heads=8, ff_dim=128, num_layers=4, dropout_rate=0.1):
    inputs = tf.keras.Input(shape=(sequence_length, input_dim))  # Input shape
    
    # Add positional encoding to inputs
    pos_encoding = get_positional_encoding(sequence_length, input_dim)
    x = tf.keras.layers.Add()([inputs, pos_encoding])  # Adding positional encoding to input
    
    # Pass through multiple Transformer encoder blocks
    for _ in range(num_layers):
        x = transformer_encoder(x, head_size=input_dim, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)
    
    # Global Average Pooling
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    # Dense layers
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(output_dim, activation='sigmoid')(x)  # Output layer with 6 emotions
    
    # Build the model
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

In [105]:
# Dataset dimensions (example values)
sequence_length = 10  # Length of each sequence (e.g., 10 frames per video)
input_dim = 515  # Number of features per timestep (e.g., face embeddings with 515 features)
output_dim = 6  # Number of classes (emotions)

# Build the model
model = build_transformer_model(sequence_length, input_dim, output_dim)

ValueError: could not broadcast input array from shape (10,258) into shape (10,257)

In [ ]:
# Summary of the model
model.summary()

# Assuming you have `X_train` and `y_train` loaded from your dataset
# Model training
history = model.fit(dataset_method_I,epochs=50, batch_size=32)